In [20]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo


# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.mars_db
collection = db.pages

url = "https://mars.nasa.gov/news/"

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')

# def init_browser():
#     executable_path = {"executable_path": "/Users/philipwewiora/Downloads/chromedriver"}
#     Browser("chrome", **executable_path, headless=False)
    


NameError: name 'html' is not defined

In [ ]:
results = soup.find_all('li', class_='slide')


for result in results:
    title = result.find("div", class_="list_text").text
    news_p = result.find("div", class_="article_teaser_body").text
    
    print('------------')
    print(article)
    print(news_p)
    
    post = {
                'title': title,
                'news_p': paragraph,
            }
    
    collection.insert_one(post)

In [21]:
# Display the MongoDB records created above
articles = db.pages.find()
for article in articles:
    print(article)

In [22]:
def scrape_info():
    browser = init_browser()

# Visit the NASA news URL
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)


    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    article = soup.find("div", class_="list_text")
    news_p = article.find("div", class_="article_teaser_body").text
    news_date = article.find("div", class_="list_date").text



In [23]:


with Browser() as browser:
    # Visit URL
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)
    # Find and click the button
    button = browser.find_by_name('button fancybox')
    # Interact with elements
    button.click()
    
    
    image = soup.find("img", class_="fancybox-image")["src"]

# Scrape the browser into soup and use soup to find the image of mars
# Save the image url to a variable called `img_url`

<img src="/spaceimages/images/mediumsize/PIA14712_ip.jpg" class="fancybox-image" style="display: inline;">




SyntaxError: invalid syntax (<ipython-input-23-237e5fdf6156>, line 16)

In [ ]:
#hand specialist = dr.peter campbell 32nd and camelback 602.393.4263